# Tapping into geothermal heat by using in a counter-flow Heat Exchanger

Inspired by <cite>Cengel, Turner: Heat Transfer</cite>

Lets A counter-flow double-pipe heat exchanger is heating water from $20~°C$ to $70~°C$
at a rate of $1.1~kg/s$. The heating is accomplished by using water from a geothermal well
available at $150~°C$ at a mass flow rate of $1.8~kg/s$. The heat exchangers inner tube has a diameter of 1.3 cm. Lets assume that the heat transfer coefficient of the heat
exchanger is $600~W/m^2°C$. What heat exchanger length is necessary
to achieve the desired heating?

![Heat Exchanger](Images/CounterflowHX.svg)

## Packages

In [62]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

## Properties

In [63]:
U = 600
D = 0.013

### Cooling water

In [64]:
mWater = 1.1
cpWater = 4.18
TinWater  = 20
ToutWater = 70

### Hot geothermal water

In [65]:
mGeo = 1.8
cpGeo = 4.31
TinGeo = 150

## 1. Determine the necessary Heat Exchanger length

### Logarithmic mean temperature difference
The logarithmic mean temperature difference $LMTD$ is defined as

\\[LMTD = \frac{\Delta T_1 - \Delta T_2}{\ln \left(\frac{\Delta T_1}{\Delta T_2}\right)}\\]


In [66]:
Q = mWater * cpWater * (ToutWater - TinWater)
ToutGeo = TinGeo - Q / (mGeo * cpGeo)

dT1 = TinGeo - ToutWater
dT2 = ToutGeo - TinWater

def LMTD(dT1,dT2):
    LMTD = (dT1 - dT2) / np.log(dT1 / dT2)
    return LMTD

A = (Q * 1000) / (U * LMTD(dT1,dT2)) #Q is given in kW
L = A / (np.pi*D)

print("The necessary Heat Exchanger length is: {length:.2f} m ".format(length = L))

The necessary Heat Exchanger length is: 104.48 m 


## Analyse heat exchanger

In [67]:
def QHX(Th1,Th2,Tc1,Tc2,U,A,strType):
    if strType == "counterflow":
        dT1 = Th1 - Tc2
        dT2 = Th2 - Tc1
    elif strType == "parallelflow":
        dT1 = Th1 - Tc1
        dT2 = Th2 - Tc2
    Q = U * A * LMTD(dT1,dT2)
    return Q

In [70]:
def analyse_HX(TinWater):
    Qcf = QHX(np.linspace(ToutWater+0.001,TinGeo+40),ToutGeo,TinWater,ToutWater,U,A,"counterflow")
    Qpf = QHX(np.linspace(TinWater+0.001,TinGeo+40),ToutGeo,TinWater,ToutWater,U,A,"parallelflow")

    plt.plot(np.linspace(ToutWater,TinGeo+40),Qcf/1000,label ="counterflow")
    plt.plot(np.linspace(TinWater,TinGeo+40),Qpf/1000,label = "parallelflow")
    
    plt.plot([TinGeo,TinGeo],[0,400],'r--')
    
    plt.xlim(0,TinGeo+40)
    plt.xlabel("$T_{in}$ hot water")
    plt.ylim(0,300)
    plt.ylabel("Q Heat transfered")
    plt.legend()

### Comparing counterflow and parallel flow heat exchanger performance

Lets compare the heat exchanger performance for counter- and parallel flow heat exchangers for different cold water inflow temperatures $T_{in}$.

In [74]:
plot = interactive(analyse_HX, TinWater=(0, 40))
plot

interactive(children=(IntSlider(value=20, description='TinWater', max=40), Output()), _dom_classes=('widget-in…